In [ ]:
# import dependencies
import pandas as pd
import datetime as dt
import pymongo

In [ ]:
# load the dataframe

file1=pd.read_csv('Business_Licenses.csv')
# file2=pd.read_csv('Sidewalk_Cafe_Permits.csv')

df1=pd.DataFrame(file1)
# df2=pd.DataFrame(file2)
df1.head()

In [ ]:

df=df1[['ACCOUNT NUMBER','DOING BUSINESS AS NAME','ADDRESS','CITY','STATE','ZIP CODE','LICENSE DESCRIPTION',
       'BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE','LICENSE TERM START DATE',
       'LICENSE TERM EXPIRATION DATE','DATE ISSUED', 'LICENSE STATUS','LATITUDE', 'LONGITUDE', 'LOCATION']]

#Select the licenses for restaurant and equivalents using the BUSINESS ACTIVITY ID
df=df.loc[(df['BUSINESS ACTIVITY ID']=='775')|
          (df['BUSINESS ACTIVITY ID']=='782') |
         (df['BUSINESS ACTIVITY ID']=='735')]

df['license_id']=df.reset_index().index

In [ ]:
df.to_csv('licenses.csv')

In [ ]:
df=pd.DataFrame(pd.read_csv('licenses.csv'))

In [ ]:
#Step 1 to get the list of active licences is to make a column for the year of expiration of the license
#Step 2 is to create a column named 'is_active' with Boolean Values (True if the expiration is 2019 or above, else False)
df['Expiration']=pd.to_datetime(df['LICENSE TERM EXPIRATION DATE']).dt.year
df['is_active']=[True if value>2018 else False for value in df['Expiration']]

In [ ]:
# Some business will have two active licenses at the same time because they may renew a license before its term
# expires. So sort the df by the Expiration date. This will mean that those expiring the earlies will be at the bottom,
# and then you can drop the duplicated and the duplicates at the bottom will be dropped
# This ensures that you aren't dropping the most recent licenses (though its not a guarantee)

active_df=df[df['is_active']==True].sort_values('Expiration',
                                                ascending=False).drop_duplicates(subset=['ACCOUNT NUMBER'])
active_df['active_id']=active_df.reset_index().index

In [ ]:

active_licenses_table=active_df[['active_id','license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID',
                                 'BUSINESS ACTIVITY','LICENSE TERM START DATE','LICENSE TERM EXPIRATION DATE']]

licenses_table=df[['license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE',
                    'DATE ISSUED']]

active_locations_table=active_df[['active_id','DOING BUSINESS AS NAME','LATITUDE','LONGITUDE','ADDRESS','CITY','STATE',
                          'ZIP CODE']]

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.restaurantsDB


In [ ]:
collection = db.active_licenses
data=active_licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [ ]:
collection=db.licenses
data=licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [ ]:
collection=db.active_locations
data=active_locations_table.to_dict(orient='records')
collection.insert_many(data)

In [ ]:
help(active_locations_table.to_dict)

In [ ]:
results = db.active_locations.find()
for result in results:
    print(result)